<a href="https://colab.research.google.com/github/Valiev-Koyiljon/Pytorch/blob/main/02_insurance_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [1]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [2]:
!pip install jovian

In [279]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [280]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file.

In [281]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [282]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [283]:
your_name = "Koyiljon Valiev"

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [284]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [285]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
1307,32,male,31.21320,4,yes,25981.699348
654,59,female,39.07200,0,no,14815.882510
1169,37,female,37.85655,1,no,7395.947069
570,31,female,32.30100,0,no,4551.163320
633,40,male,25.20255,2,no,8679.765540


Let us answer some basic questions about the dataset.


**Q1: How many rows does the dataset have?**

In [286]:
num_rows = dataframe.shape[0]
print(num_rows)

1271


**Q2: How many columns doe the dataset have**

In [287]:
num_cols = dataframe.shape[1]
print(num_cols)

6


**Q3: What are the column titles of the input variables?**

In [288]:
input_cols = dataframe[['age', 'sex', 'bmi', 'children', 'smoker']].columns
input_cols


Index(['age', 'sex', 'bmi', 'children', 'smoker'], dtype='object')

**Q4: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [289]:
categorical_cols = ["sex", "smoker"]

**Q5: What are the column titles of output/target variable(s)?**

In [290]:
output_cols = ["charges"]
output_cols

['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ai/aakashns/dataviz-cheatsheet

In [291]:
output_cols_min  = dataframe[output_cols].min()
output_cols_min

charges    1357.467419
dtype: float64

In [292]:
output_cols_max  = dataframe[output_cols].max()
output_cols_max

charges    77162.217892
dtype: float64

In [293]:
output_cols_aver  = dataframe[output_cols].mean()
output_cols_aver

charges    16142.696816
dtype: float64

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [294]:
!pip install jovian --upgrade -q

In [295]:
import jovian

In [296]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [297]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [298]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[32.     ,  1.     , 31.2132 ,  4.     ,  1.     ],
        [59.     ,  0.     , 39.072  ,  0.     ,  0.     ],
        [37.     ,  0.     , 37.85655,  1.     ,  0.     ],
        ...,
        [57.     ,  0.     , 22.311  ,  1.     ,  0.     ],
        [23.     ,  1.     , 38.184  ,  0.     ,  0.     ],
        [57.     ,  0.     , 26.6178 ,  1.     ,  0.     ]]),
 array([[25981.699348 ],
        [14815.88251  ],
        [ 7395.9470695],
        ...,
        [14559.11446  ],
        [ 2210.48003  ],
        [26852.8489031]]))

In [299]:
import numpy as np
np.isnan(targets_array).sum() # no null value

0

**Q6: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [300]:
inputs = torch.tensor(inputs_array, dtype=torch.float32)
targets = torch.tensor(targets_array, dtype=torch.float32)

In [301]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [302]:
dataset = TensorDataset(inputs, targets)

**Q7: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [303]:
val_percent = 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size])
 # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q8: Pick a batch size for the data loader.**

In [304]:
batch_size = 64

In [305]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [306]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[35.0000,  0.0000, 39.8046,  2.0000,  0.0000],
        [18.0000,  1.0000, 39.0720,  1.0000,  0.0000],
        [41.0000,  1.0000, 26.5734,  1.0000,  0.0000],
        [21.0000,  1.0000, 28.5770,  2.0000,  0.0000],
        [40.0000,  1.0000, 36.3802,  1.0000,  1.0000],
        [31.0000,  1.0000, 31.7404,  1.0000,  0.0000],
        [42.0000,  1.0000, 34.6931,  0.0000,  0.0000],
        [33.0000,  1.0000, 27.5224,  0.0000,  1.0000],
        [47.0000,  1.0000, 40.0488,  1.0000,  1.0000],
        [33.0000,  1.0000, 39.6825,  2.0000,  0.0000],
        [22.0000,  0.0000, 26.9730,  0.0000,  0.0000],
        [25.0000,  1.0000, 29.6315,  4.0000,  0.0000],
        [51.0000,  0.0000, 45.1326,  0.0000,  0.0000],
        [28.0000,  0.0000, 32.0568,  1.0000,  0.0000],
        [28.0000,  0.0000, 29.4261,  2.0000,  0.0000],
        [39.0000,  0.0000, 26.8897,  5.0000,  0.0000],
        [44.0000,  0.0000, 43.2345,  0.0000,  1.0000],
        [51.0000,  1.0000, 24.8862,  0.0000,  0.0000],
  

Let's save our work by committing to Jovian.

In [307]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment).


In [308]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q9: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [309]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)  # fill this (hint: use input_size & output_size defined above)

    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out

    def training_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calcuate loss
        loss = F.mse_loss(out, targets)                   # fill this
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.mse_loss(out, targets)
        return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [310]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [311]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2868,  0.0502,  0.2633, -0.1826, -0.1674]], requires_grad=True),
 Parameter containing:
 tensor([0.2324], requires_grad=True)]

One final commit before we train the model.

In [312]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [313]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q10: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [314]:
# model = InsuranceModel()

result = evaluate(model, val_loader)  # Use the evaluate function
print(result)


{'val_loss': 504676288.0}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q11: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [315]:
epochs = 10
lr = 1e-2
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [10], val_loss: nan


In [316]:
model = InsuranceModel()

epochs = 100
lr = 1e-3
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [60], val_loss: nan
Epoch [80], val_loss: nan
Epoch [100], val_loss: nan


In [317]:
model = InsuranceModel()

epochs = 1000
lr = 1e-5
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 199505312.0000
Epoch [40], val_loss: 199443840.0000
Epoch [60], val_loss: 199123616.0000
Epoch [80], val_loss: 199025328.0000
Epoch [100], val_loss: 198577296.0000
Epoch [120], val_loss: 198061664.0000
Epoch [140], val_loss: 197851696.0000
Epoch [160], val_loss: 197785440.0000
Epoch [180], val_loss: 197274592.0000
Epoch [200], val_loss: 196959808.0000
Epoch [220], val_loss: 196907136.0000
Epoch [240], val_loss: 196302656.0000
Epoch [260], val_loss: 196175520.0000
Epoch [280], val_loss: 195861904.0000
Epoch [300], val_loss: 195481792.0000
Epoch [320], val_loss: 195360992.0000
Epoch [340], val_loss: 194910944.0000
Epoch [360], val_loss: 194481504.0000
Epoch [380], val_loss: 194400864.0000
Epoch [400], val_loss: 194005872.0000
Epoch [420], val_loss: 193643648.0000
Epoch [440], val_loss: 193399616.0000
Epoch [460], val_loss: 193207040.0000
Epoch [480], val_loss: 192905984.0000
Epoch [500], val_loss: 192889904.0000
Epoch [520], val_loss: 192234800.0000
Epoch [540], val

In [318]:
model = InsuranceModel()

epochs =1000
lr = 1e-6
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 211096416.0000
Epoch [40], val_loss: 200480656.0000
Epoch [60], val_loss: 199731872.0000
Epoch [80], val_loss: 199623904.0000
Epoch [100], val_loss: 199580128.0000
Epoch [120], val_loss: 199576608.0000
Epoch [140], val_loss: 199564096.0000
Epoch [160], val_loss: 199531104.0000
Epoch [180], val_loss: 199515776.0000
Epoch [200], val_loss: 199501664.0000
Epoch [220], val_loss: 199484432.0000
Epoch [240], val_loss: 199472288.0000
Epoch [260], val_loss: 199431456.0000
Epoch [280], val_loss: 199431184.0000
Epoch [300], val_loss: 199408768.0000
Epoch [320], val_loss: 199367248.0000
Epoch [340], val_loss: 199366720.0000
Epoch [360], val_loss: 199338528.0000
Epoch [380], val_loss: 199305392.0000
Epoch [400], val_loss: 199288704.0000
Epoch [420], val_loss: 199281136.0000
Epoch [440], val_loss: 199241456.0000
Epoch [460], val_loss: 199215712.0000
Epoch [480], val_loss: 199192464.0000
Epoch [500], val_loss: 199173088.0000
Epoch [520], val_loss: 199154560.0000
Epoch [540], val

In [319]:
model = InsuranceModel()

epochs = 10000
lr = 1e-4  # this learning rate is fitted well to decrease the loss
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 196469072.0000
Epoch [40], val_loss: 194836256.0000
Epoch [60], val_loss: 190722880.0000
Epoch [80], val_loss: 207326336.0000
Epoch [100], val_loss: 185185424.0000
Epoch [120], val_loss: 185243664.0000
Epoch [140], val_loss: 180007744.0000
Epoch [160], val_loss: 178262160.0000
Epoch [180], val_loss: 176898688.0000
Epoch [200], val_loss: 172546304.0000
Epoch [220], val_loss: 170318240.0000
Epoch [240], val_loss: 175479472.0000
Epoch [260], val_loss: 168161200.0000
Epoch [280], val_loss: 166940944.0000
Epoch [300], val_loss: 161353120.0000
Epoch [320], val_loss: 158781888.0000
Epoch [340], val_loss: 158598672.0000
Epoch [360], val_loss: 154713472.0000
Epoch [380], val_loss: 153422592.0000
Epoch [400], val_loss: 150535376.0000
Epoch [420], val_loss: 152356224.0000
Epoch [440], val_loss: 150135712.0000
Epoch [460], val_loss: 144867200.0000
Epoch [480], val_loss: 144507792.0000
Epoch [500], val_loss: 144959552.0000
Epoch [520], val_loss: 139848112.0000
Epoch [540], val

**Q12: What is the final validation loss of your model?**

In [328]:
val_loss =54228080.0000

Let's log the final validation loss to Jovian and commit the notebook

In [329]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [330]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q13: Complete the following function definition to make predictions on a single input**

In [332]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)



# Assuming you have a single input tensor 'input', a target value 'target', and a model 'model'


In [343]:

input, target = val_ds[99]
predict_single(input, target, model)

Input: tensor([19.0000,  0.0000, 31.0023,  3.0000,  0.0000])
Target: tensor([22794.8320])
Prediction: tensor([4782.3477])


In [342]:
input, target = val_ds[8]
predict_single(input, target, model)

Input: tensor([19.0000,  0.0000, 19.7580,  0.0000,  0.0000])
Target: tensor([2090.6199])
Prediction: tensor([1331.0713])


In [336]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([53.0000,  1.0000, 31.7460,  3.0000,  0.0000])
Target: tensor([13616.6396])
Prediction: tensor([13912.7480])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog.

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [344]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.
[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.
